In [1]:
import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

In [2]:
#import dotenv
import pydot
import requests
import numpy as np
import pandas as pd
import ctypes
import shutil
import multiprocessing
import multiprocessing.sharedctypes as sharedctypes
import os.path
import ast

In [3]:
import librosa
import librosa.display

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [5]:
data_root = tf.keras.utils.get_file('Audio', 'https://os.unil.cloud.switch.ch/fma/fma_small.zip' , extract=True)

7679594875/7679594875 [==============================] - 323s 0us/step


In [6]:
!cp -r ~/.keras/datasets/fma_small /content

In [7]:
meta_data = tf.keras.utils.get_file('Meta' , 'https://os.unil.cloud.switch.ch/fma/fma_metadata.zip' , extract=True)

358412441/358412441 [==============================] - 16s 0us/step


In [8]:
!cp ~/.keras/datasets/fma_metadata/tracks.csv /content

In [9]:
!cp ~/.keras/datasets/fma_metadata/genres.csv /content

In [10]:
!cp ~/.keras/datasets/fma_metadata/echonest.csv /content

In [11]:
!cp ~/.keras/datasets/fma_metadata/features.csv /content

In [12]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks

In [13]:
tracks = load('/content/tracks.csv')
features = load('/content/features.csv')

np.testing.assert_array_equal(features.index, tracks.index)

tracks.shape, features.shape

subset = tracks.index[tracks['set', 'subset'] <= 'small']

tracks = tracks.loc[subset]
features_all = features.loc[subset]

tracks.shape, features_all.shape

((8000, 52), (8000, 518))

In [15]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

6400 training examples, 800 validation examples, 800 testing examples
Top genres (8): ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']
All genres (114): [1, 2, 6, 10, 12, 15, 16, 17, 18, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 38, 41, 42, 45, 46, 47, 49, 53, 58, 64, 66, 70, 71, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 167, 171, 172, 174, 177, 180, 181, 182, 183, 184, 185, 186, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 314, 337, 359, 360, 361, 362, 400, 401, 404, 439, 440, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 695, 741, 763, 808, 811, 1032, 1060, 1193, 1235]


In [16]:
def pre_process(tracks, features, columns, verbose=False):
    # Каждому жанру присвоить число
    enc = LabelEncoder()
    labels = tracks['track', 'genre_top']

    # Разделение на тренировочную, валидационную и тестовую выборки
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
    X_train = features.loc[train, columns]
    X_val = features.loc[val, columns]
    X_test = features.loc[test, columns]
    
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    
    # стандартизация
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test

In [17]:
def test_classifiers_features(classifiers, feature_sets):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features_all, fset)
        scores.loc[fset_name, 'dim'] = X_train.shape[1]
        for clf_name, clf in classifiers.items():  # tqdm_notebook(classifiers.items(), desc='classifiers', leave=False):
            t = time.process_time()
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            scores.loc[fset_name, clf_name] = score
            times.loc[fset_name, clf_name] = time.process_time() - t
    return scores, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [18]:
features.columns.levels[0]

Index(['chroma_cens', 'chroma_cqt', 'chroma_stft', 'mfcc', 'rmse',
       'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast',
       'spectral_rolloff', 'tonnetz', 'zcr'],
      dtype='object', name='feature')

In [28]:
classifiers = {
    'LR': LogisticRegression(),
    'kNN': KNeighborsClassifier(n_neighbors=50),
    'SVC': SVC(),
}

feature_sets = {}
for name in features.columns.levels[0]:
    feature_sets[name] = name

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

<ipython-input-17-58cfddee37e8>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):


features:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,dim,LR,kNN,SVC
chroma_cens,84.000000,25.00%,22.62%,32.00%
chroma_cqt,84.000000,27.38%,24.88%,29.25%
chroma_stft,84.000000,33.25%,31.50%,37.62%
mfcc,140.000000,42.12%,35.75%,46.38%
rmse,7.000000,21.12%,19.50%,23.12%
spectral_bandwidth,7.000000,31.87%,30.25%,31.37%
spectral_centroid,7.000000,30.88%,29.62%,33.12%
spectral_contrast,49.000000,36.12%,34.88%,40.00%
spectral_rolloff,7.000000,28.38%,30.50%,31.50%
tonnetz,42.000000,26.75%,23.25%,27.50%


,LR,kNN,SVC
chroma_cens,1.2068,0.3483,5.9520
chroma_cqt,1.1398,0.3271,7.1504
chroma_stft,1.1736,0.3202,5.7357
mfcc,1.9109,0.3085,7.7350
rmse,0.5812,0.2283,2.7539
spectral_bandwidth,0.5128,0.2052,2.6678
spectral_centroid,0.5748,0.1771,3.9329
spectral_contrast,1.0169,0.3106,3.6806
spectral_rolloff,0.5930,0.2086,2.5671
tonnetz,0.8443,0.3385,5.7894
